In [1]:
import numpy as np
import h5py

In [2]:
hf = h5py.File('/content/drive/MyDrive/data/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5','r')
x = hf.get('X')[:50000]
y = hf.get('y')[:50000]

x_electron = np.asarray(x)
y_electron = np.asarray(y)
x = None
y = None

In [3]:
from keras.optimizers.schedules.learning_rate_schedule import NoisyLinearCosineDecay
hf1 = h5py.File('/content/drive/MyDrive/data/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5','r')
x1 = hf1.get('X')[:50000]
y1 = hf1.get('y')[:50000]

x_photon = np.asarray(x1)
y_photon = np.asarray(y1)
x1 = None
y1 = None

In [4]:
x_data = np.concatenate((x_electron,x_photon),axis=0)
y_data = np.concatenate((y_electron,y_photon),axis=0)
avg_channel = np.mean(x_data[:,:, :, :2], axis=-1, keepdims=True)

# Concatenate the average channel with the original image
x_data = np.concatenate((x_data, avg_channel), axis=-1)

In [5]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
x_train, x_validate, y_train, y_validate = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_validate, y_validate, test_size=0.5, random_state=42)

x_data = None
y_data = None

In [8]:
import tensorflow as tf

input_shape = (32, 32, 3)
num_classes = 1

def residual_block(inputs, filters):
    x = tf.keras.layers.Conv2D(filters, 3, padding='same', activation='relu')(inputs)
    x = tf.keras.layers.Conv2D(filters, 3, padding='same', activation=None)(x)
    if inputs.shape[-1] != filters:
        inputs = tf.keras.layers.Conv2D(filters, 1, padding='same', activation=None)(inputs)
    x = tf.keras.layers.add([x, inputs])
    return tf.keras.layers.Activation('relu')(x)

inputs = tf.keras.layers.Input(shape=input_shape)
x = tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu')(inputs)
x = residual_block(x, 16)
x = residual_block(x, 16)
x = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)(x)
x = residual_block(x, 32)
x = residual_block(x, 32)
x = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
outputs = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)

model1 = tf.keras.models.Model(inputs=inputs, outputs=outputs)
model1.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_10 (Conv2D)             (None, 32, 32, 16)   448         ['input_2[0][0]']                
                                                                                                  
 conv2d_11 (Conv2D)             (None, 32, 32, 16)   2320        ['conv2d_10[0][0]']              
                                                                                                  
 conv2d_12 (Conv2D)             (None, 32, 32, 16)   2320        ['conv2d_11[0][0]']              
                                                                                            

In [9]:
model1.compile(loss='binary_crossentropy',
              optimizer='nadam',
              metrics=['accuracy'])
model1.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=20)

Epoch 1/20
2500/2500 [==============================] - 49s 11ms/step - loss: 0.6547 - accuracy: 0.6145 - val_loss: 0.6435 - val_accuracy: 0.6265
Epoch 2/20
2500/2500 [==============================] - 27s 11ms/step - loss: 0.6279 - accuracy: 0.6524 - val_loss: 0.6074 - val_accuracy: 0.6752
Epoch 3/20
2500/2500 [==============================] - 28s 11ms/step - loss: 0.5988 - accuracy: 0.6857 - val_loss: 0.5887 - val_accuracy: 0.6922
Epoch 4/20
2500/2500 [==============================] - 27s 11ms/step - loss: 0.5840 - accuracy: 0.7003 - val_loss: 0.5817 - val_accuracy: 0.7004
Epoch 5/20
2500/2500 [==============================] - 28s 11ms/step - loss: 0.5782 - accuracy: 0.7053 - val_loss: 0.5792 - val_accuracy: 0.6996
Epoch 6/20
2500/2500 [==============================] - 26s 11ms/step - loss: 0.5716 - accuracy: 0.7103 - val_loss: 0.5768 - val_accuracy: 0.7044
Epoch 7/20
2500/2500 [==============================] - 26s 10ms/step - loss: 0.5681 - accuracy: 0.7133 - val_loss: 0.5804 -

In [10]:
from sklearn.metrics import roc_auc_score
pred_prob1 = model1.predict(x_test)
auc_score1 = roc_auc_score(y_test, pred_prob1[:])
auc_score1

313/313 [==============================] - 1s 3ms/step


0.7813479721317492

In [11]:
import tensorflow as tf

input_shape = (32, 32, 3)
num_classes = 1

def residual_block(inputs, filters):
    x = tf.keras.layers.Conv2D(filters, 3, padding='same', activation='relu')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(filters, 3, padding='same', activation=None)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    if inputs.shape[-1] != filters:
        inputs = tf.keras.layers.Conv2D(filters, 1, padding='same', activation=None)(inputs)
        inputs = tf.keras.layers.BatchNormalization()(inputs)
    x = tf.keras.layers.add([x, inputs])
    return tf.keras.layers.Activation('relu')(x)

inputs = tf.keras.layers.Input(shape=input_shape)
x = tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu')(inputs)
x = tf.keras.layers.BatchNormalization()(x)
x = residual_block(x, 32)
x = residual_block(x, 32)
x = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)(x)
x = residual_block(x, 64)
x = residual_block(x, 64)
x = tf.keras.layers.MaxPooling2D(pool_size=2, strides=2)(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)

model2 = tf.keras.models.Model(inputs=inputs, outputs=outputs)
model2.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_20 (Conv2D)             (None, 32, 32, 32)   896         ['input_3[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 32)  128         ['conv2d_20[0][0]']              
 alization)                                                                                       
                                                                                                  
 conv2d_21 (Conv2D)             (None, 32, 32, 32)   9248        ['batch_normalization[0][0]

In [12]:
from keras.optimizers import Adam

model2.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.009),
              metrics=['accuracy'])
model2.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=20)

Epoch 1/20
2500/2500 [==============================] - 27s 10ms/step - loss: 0.8181 - accuracy: 0.5881 - val_loss: 0.6537 - val_accuracy: 0.6095
Epoch 2/20
2500/2500 [==============================] - 25s 10ms/step - loss: 0.6352 - accuracy: 0.6543 - val_loss: 0.6343 - val_accuracy: 0.6733
Epoch 3/20
2500/2500 [==============================] - 25s 10ms/step - loss: 0.6129 - accuracy: 0.6826 - val_loss: 0.8784 - val_accuracy: 0.4993
Epoch 4/20
2500/2500 [==============================] - 24s 10ms/step - loss: 0.6013 - accuracy: 0.6913 - val_loss: 0.6171 - val_accuracy: 0.6695
Epoch 5/20
2500/2500 [==============================] - 25s 10ms/step - loss: 0.5955 - accuracy: 0.6978 - val_loss: 0.6079 - val_accuracy: 0.6773
Epoch 6/20
2500/2500 [==============================] - 25s 10ms/step - loss: 0.5915 - accuracy: 0.6990 - val_loss: 0.5807 - val_accuracy: 0.6974
Epoch 7/20
2500/2500 [==============================] - 25s 10ms/step - loss: 0.5890 - accuracy: 0.7009 - val_loss: 0.5875 -

In [13]:
from sklearn.metrics import roc_auc_score
pred_prob2 = model2.predict(x_test)
auc_score2 = roc_auc_score(y_test, pred_prob2[:])
auc_score2

313/313 [==============================] - 1s 4ms/step


0.7849721247491227

In [15]:
pred_final = 0.6*auc_score1 + 0.4*auc_score2
pred_final

0.7827976331786985